In [ ]:
%load_ext nb_unittest

# Test Bank

This is an example test bank.

In [ ]:
## Code shared by the test bank can go here.

def shared():
    pass

## The Simplest Question

Questions are specialized Python classes that include the question text, some information about what kind of solution is expected and test cases. To enable maximum reusability the variable `self.solution_cell` will be defined by the framework code. In the simplest form, questions can expect:

1. The solution will be marked with a tag that has the same name as the test class (e.g. `ArgumentReverser`) 
2. `self.solution_cell` will contain the `TagCacheEntry` with the solution cell. 

Here is a very simple test question:

In [ ]:
from nbquiz.question import TestQuestion

class SimpleArgReverse(TestQuestion):
    """
    Write a function called `arg_reverser` that takes three arguments and returns
    them in reverse order.
    """

    def test_1_arg_reverser(self):
        """Testing arg_reverser()"""

        assert "arg_reverser" in self.solution_cell.ns, """I can't find your solution."""
        self.assertEqual(self.solution_cell.ns["arg_reverser"](1, 2, 3), (3, 2, 1))
        self.assertEqual(
            self.solution_cell.ns["arg_reverser"]("one", "two", "three"), ("three", "two", "one")
        )

Generate a preview for copy/paste to an LMS

In [ ]:
from IPython.display import Markdown
Markdown(SimpleArgReverse.question())

### Question Self Tests

To test the question you can check it against the correct and incorrect solutions:

In [ ]:
"""@SimpleArgReverse"""

def arg_reverser(arg1, arg2, arg3):
    """Reverse my arguments."""
    return arg3, arg2, arg1

In [ ]:
%%testing SimpleArgReverse
nbtest_cases = [SimpleArgReverse]

In [ ]:
import nb_unittest
nb_unittest.assert_ok()

What about tests versus invalid solutions? They should be able to be in the notebook:

In [ ]:
"""@SimpleArgReverse"""

def arg_reverser(a, b):
    """Wrong number of arguments."""
    return b, a

In [ ]:
%%testing SimpleArgReverse
nbtest_cases = [SimpleArgReverse]

In [ ]:
nb_unittest.assert_error()

Incorrect return values are reported:

In [ ]:
"""@SimpleArgReverse"""

def arg_reverser(a, b, c):
    """Bad return value"""
    return b, a, c

In [ ]:
%%testing SimpleArgReverse
nbtest_cases = [SimpleArgReverse]

In [ ]:
nb_unittest.assert_error()

## Using FunctionQuestion 

The `FunctionQuestion` class supports a number of ways to validate a function question. Function questions can also become cell questions. See below. A function question:

1. Validates that the function exists and is a function. 
1. Ensures that the function has a docstring. 
1. Validates the number, order and names of arguments. 
1. Validates the return type by wrapping the student function. 

A simple function question is shown below:

In [ ]:
import ast 
from nbquiz.question import FunctionQuestion

class TriangleMaxArea1(FunctionQuestion):
    """
    Write a function called `triangle_limit` that takes two arguments, `base` and `height`. 
    Calculate the area of a triangle with `base` and `height`. If the area of the triangle 
    is greater than or equal to `limit`, return `True` otherwise return `False`. 
    """

    tokens_required = [ast.GtE]
    tokens_forbidden = [ast.If]

    # self.solution will be this attribute
    name = "triangle_limit" 

    # Validate self.solution based on the type hints style attributes. 
    # When a FunctionValidator is used `self.solution` will be the function.
    annotations = {"base": float, "height": float, "return": bool}

    def test_the_func(self):
        self.assertEqual(
            self.solution(10, 10), True, """Should be True."""
        )
        self.assertEqual(
            self.solution(10,1), False, """Should be True."""
        )
        self.assertEqual(
            self.solution(10,0.1), False, """Should be False."""
        )

In [ ]:
print(TriangleMaxArea1.question())

In [ ]:
"""@TriangleMaxArea1"""
def triangle_limit(base, height):
    """This is my solution."""
    return (base * height) / 2 >= 10

In [ ]:
%%testing TriangleMaxArea1
nbtest_cases = [TriangleMaxArea1]

A number of error conditions are checked for:

In [ ]:
"""@TriangleMaxArea1"""

# Wrong or misspelled name. 
def triangle_limit_blah(base, height):
    """This is my solution."""
    return (base * height) / 2 >= 10

In [ ]:
%%testing TriangleMaxArea1
nbtest_cases = [TriangleMaxArea1]

In [ ]:
"""@TriangleMaxArea1"""

# Parameters are incorrect or are in the wrong order:
def triangle_limit(height, base):
    """This is my solution."""
    return (base * height) / 2 >= 10

In [ ]:
%%testing TriangleMaxArea1
nbtest_cases = [TriangleMaxArea1]

In [ ]:
"""@TriangleMaxArea1"""

# No docstring
def triangle_limit(base, height):
    return (base * height) / 2 >= 10

In [ ]:
%%testing TriangleMaxArea1
nbtest_cases = [TriangleMaxArea1]

In [ ]:
"""@TriangleMaxArea1"""

# Wrong return type.
def triangle_limit(base, height):
    """This is my solution."""
    return (base >= height) / 2 

In [ ]:
%%testing TriangleMaxArea1
nbtest_cases = [TriangleMaxArea1]

In [ ]:
"""@TriangleMaxArea1"""

# Missing the ast.GtE token.
def triangle_limit(base, height):
    """This is my solution."""
    return (base * height) / 2 <= 10

In [ ]:
%%testing TriangleMaxArea1
nbtest_cases = [TriangleMaxArea1]

In [ ]:
"""@TriangleMaxArea1"""

# Uses the ast.If token
def triangle_limit(base, height):
    """This is my solution."""
    if (base * height) / 2 >= 10:
        return True
    else: 
        return False

In [ ]:
%%testing TriangleMaxArea1
nbtest_cases = [TriangleMaxArea1]

## Parameterized Function Questions

Questions can be parameterized so that variants of the question can be created automatically. The previous version of `TriangleMaxArea` hard coded the function name, arguments and the constant `10`. This version parameterizes them using class variables. Care should be taken to ensure test cases work for all parameters. 

The special form of `{variable}` in the `annotations` dictionary and the docstring causes the validator to look up the function name in the class variable called `variable`. 

> NOTE: These are **not** f-strings. The `{{variable}}` is a literal.

In [ ]:
import ast 
from nbquiz.question import FunctionQuestion, Parameter

class TriangleMaxArea2(FunctionQuestion):
    """
    Write a function called {{name}} that takes two arguments, {{base}} and
    {{height}}. Calculate the area of a triangle with {{base}} and {{height}}.
    If the area of the triangle is greater than or equal to {{limit}}, return
    `True` otherwise return `False`. 

    Example:

    ```python
    >>> {{name|plain}}(0, 0)
    0
    ```
    """

    tokens_required = [ast.GtE]
    tokens_forbidden = [ast.If]

    # self.solution will be this attribute
    name: str = "triangle_limit" 
    base: str = "base"
    height: str = "height"
    limit: int = 10

    # Validate self.solution based on the type hints style attributes. 
    # When a FunctionValidator is used `self.solution` will be the function.
    annotations = {"{base}": float, "{height}": float, "return": bool}

    def test_the_func(self):
        self.assertEqual(
            self.solution(self.limit, self.limit), True, f"""Should {self.limit} x 2 should be True."""
        )
        self.assertEqual(
            self.solution(self.limit, 1), False, f"""Should {self.limit}, 1 be True."""
        )
        self.assertEqual(
            self.solution(self.limit/10, self.limit/10), False, f"""Should {self.limit}, 1/10 be False."""
        )

In [ ]:
Markdown(TriangleMaxArea2.question())

In [ ]:
"""@TriangleMaxArea2"""
def triangle_limit(base, height):
    """This is my solution."""
    return (base * height) / 2 >= 10

In [ ]:
%%testing TriangleMaxArea2
nbtest_cases = [TriangleMaxArea2]

### Creating a Variant 

When a question class is parameterized you can use the `variant` class function to create variants of the question:


In [ ]:
variant = TriangleMaxArea2.variant(
    name = "triangle_max_100",
    base = "b",
    height = "h",
    limit = 100, 
)

In [ ]:
Markdown(variant.question())

Variants should normally be used inside of a `QuestionGroup` (see below) but can be referenced by an automatically assigned a mangled class name that follows the formula:

```
BaseClassName_param1:value1_param2:value2
```

In [ ]:
variant.__name__

In [ ]:
"""@TriangleMaxArea2_name:triangle_max_100_base:b_height:h_limit:100"""

def triangle_max_100(b, h):
    """Some question"""
    return (b * h) / 2 >= 100

In [ ]:
%%testing variant
nbtest_cases = [variant]

## Making Cell Variants

A `FunctionQuestion` can easily become a cell-based question for tests that happen before students learn function syntax. The `annotations` dictionary is used to create variables in the cell namespace that work just like function arguments.  

### A Cell Variant 

A question that inherits from a FunctionQuestion and a CellQuestion can be used to test a cell using the `run()` method of a `CellCacheEntry`.  

In [ ]:
from nbquiz.question import CellQuestion

class TriangleMaxArea1Cell(TriangleMaxArea1, CellQuestion):
    """
    Make a cell that creates two variables, `base` and `height`. Calculate the area of a triangle with `base` and `height`. 
    If the area of the triangle is greater than or equal to 10, the cell should result in `True`, `False` otherwise. 
    """

In [ ]:
Markdown(TriangleMaxArea1Cell.question())

In [ ]:
"""@TriangleMaxArea1Cell"""

base = 10 
height = 100

(base * height) / 2 >= 10

Parameterized `FunctionTests` are even better: 

In [ ]:
%%testing TriangleMaxArea1Cell
nbtest_cases = [TriangleMaxArea1Cell]

In [ ]:
from nbquiz.question import CellQuestion

class TriangleMaxArea2Cell(TriangleMaxArea2, CellQuestion):
    """
    Make a cell that creates two variables, {{base}} and {{height}}. Calculate
    the area of a triangle with {{base}} and {{height}}. If the area of the
    triangle is greater than or equal to {{limit}}, the cell should result in
    `True`, `False` otherwise. 
    """

In [ ]:
Markdown(TriangleMaxArea2Cell.question())

In [ ]:
"""@TriangleMaxArea2Cell"""

base = 10 
height = 100

(base * height) / 2 >= 10


In [ ]:
%%testing TriangleMaxArea2Cell
nbtest_cases = [TriangleMaxArea2Cell]

Cell based variants work too:

In [ ]:
variant = TriangleMaxArea2Cell.variant(
    base = "bob",
    height = "hillary",
    limit = 100,

)

In [ ]:
Markdown(variant.question())

In [ ]:
"""@TriangleMaxArea2Cell_base:bob_height:hillary_limit:100"""

bob = 100 
hillary = 100 

(bob * hillary) / 2 >= 100 

In [ ]:
%%testing variant
nbtest_cases = [variant]

## Question Groups 

When a test bank is exported to Canvas question groups become a question group
in the assessment. During the quiz, students receive one or more of the
questions in a group at random. Question groups ensure that every student
receives a unique test making certain kinds of cheating harder. Question groups
are best used to:

1. Group question variants that differ only by a constant or name. 
1. Group questions of a similar difficulty and topic for test randomization. 

The `QuestionGroup` class is just a wrapper around a `list`, but is special
because it's recognized by the framework as belonging to the test bank. 

In [ ]:
from nbquiz.question import QuestionGroup

TriangleMaxArea2Group = QuestionGroup()
for i in range(30,80,5):
    TriangleMaxArea2Group.append(TriangleMaxArea2.variant(limit=i))

You can use the initializer too:

In [ ]:
TriangleMaxArea2CellGroup = QuestionGroup("TriangleMaxArea2CellGroup", [
    TriangleMaxArea2Cell.variant(limit=i) for i in range(30,80,5)
])

The test bank loader will assign a tag to groups based on their class names. When generating a test you'll use `@TriangleMaxArea2CellGroup` and `@TriangleMaxArea2CellGroup` to refer to these groups. The name of a question group will appear in the test to instructors only.

## Testing Markdown Extensions 

Mardown math syntax is largely supported because they seem to render properly without the need for JavaScript or CSS. However, it's awkward to specify LaTeX because the docstring is first passed to `format()`, requiring the doubling of `{` and `}` also Python will eat the `\` characters unless you make the docstring a `r` string.

In [ ]:
class MathQuestion(TestQuestion):
    r"""
    Test the math extensions: $y = mx + b$

    $$
    e = mc^2
    $$ 

    Hence, for $\alpha \in (0, 1)$,
    
    $$
    \mathbb P (\alpha \bar{X} \ge \mu) \le \alpha;
    $$

    i.e., $[\alpha \bar{X}, \infty)$ is a lower 1-sided $1-\alpha$ confidence bound for $\mu$.    

    \begin{align}
    a_{11}& =b_{11}&
    a_{12}& =b_{12}\\
    a_{21}& =b_{21}&
    a_{22}& =b_{22}+c_{22}
    \end{align}
    """


In [ ]:
Markdown(MathQuestion.question())